<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


# 관련 라이브러 및 함수 정의

## 라이브러리 로드

In [1]:
# 기초 전처리
import pandas as pd
import numpy as np
import geopandas as gpd
# from tqdm import tqdm
%matplotlib inline

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import os

# 캔버스 사이즈 적용
plt.rcParams['figure.figsize'] = (12, 9)

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
# 지도 시각화
import folium
from folium import plugins

## 필요 함수 로드

### 데이타 테이블 처리 함수

In [3]:
def get_str(x):
    return str(x)

def get_object_type(df):
    df['상권_코드'] = df['상권_코드'].apply(get_str)
    df.astype({'상권_코드':object})
    try:
        df.drop('상권_이름', inplace=True, axis=1)
    except:
        pass
    return df

In [4]:
def get_base(only_골목=True):
    bs_area = gpd.read_file('data/geojson/상권_geometry_좌표.geojson', driver= 'GeoJSON')
    if only_골목:
        bs_area= bs_area[bs_area['TRDAR_SE_C'] == 'A']

    df_base = bs_area[['TRDAR_CD','TRDAR_SE_1']].sort_values('TRDAR_CD').reset_index(drop=True)
    df_base.columns = ['상권_코드', '상권_이름']
    df_base = get_object_type(df_base)
    return df_base

### 상권 시각화 코드

In [5]:
# 리스트 담은 상권 코드 시각화

def vis_by_selected_상권_코드(selected_list_상권코드):
    bs_cd = bs_area[bs_area['TRDAR_CD'].isin(selected_list_상권코드)]
    data = np.array(bs_cd[['y','x']])
    popups1 = bs_cd['TRDAR_CD_N'].to_list()
    popups2 = bs_cd['TRDAR_SE_1'].to_list()

    popups = [a + '_' + b for a, b in zip(popups1, popups2)]
    m = folium.Map(location=[37.52915424294603, 126.98045169603408],
    #                tiles='Stamen Toner',
    #                tiles='Stamen Watercolor',
    #                tiles='Stamen Terrain',
                   zoom_start=11)

    folium.Choropleth( geo_data=seoul_lay,
                       fill_color=cmaps3[0],
                       fill_opacity=0.2,
                       line_opacity=0.5).add_to(m)

    for idx, se in  enumerate(bs_area['TRDAR_SE_C'].unique()):
        folium.Choropleth( geo_data=bs_area[bs_area['TRDAR_SE_C']==se],
                           fill_color=cmaps3[idx+1],
                           fill_opacity=0.5,
                           line_opacity=0.3 ).add_to(m)

    plugins.MarkerCluster(data, popups=popups).add_to(m)

    folium.LayerControl().add_to(m)
    
    return m

# 기본 데이터 로드

## 서울시 골목상권 및 상권 관련 자료 리스트

* 골목상권 분석 사이트 : https://golmok.seoul.go.kr/main.do

**용어 정리**

* 배후지란? 상권 영역으로 부터 200m 안

1. 상권-추정매출(2014~2021_2분기) /배후지 추정매출(동일) : 

2. 생활인구 *상권/배후지* : 

6. 상주인구 *상권/배후지* :

7. 직장인구 *상권/배후지* : 

3. 점포 *상권/배후지* :

4. 아파트 *상권/배후지* :

5. 집객시설 *상권/배후지* :

8. 소득소비 *배후지* :

9. 상권별 생존률 :  https://golmok.seoul.go.kr/businessAreaAnalysis.do

## 상권 지리 테이블 로드

In [6]:
bs_area = gpd.read_file('data/geojson/상권_geometry_좌표.geojson', driver= 'GeoJSON')

In [7]:
url='https://raw.githubusercontent.com/suanlab/dataset/master'
seoul_geo = f'{url}/seoul_municipalities_geo_simple.json'
seoul_lay = gpd.read_file(seoul_geo)

In [8]:
bs_area.head(2)

,TRDAR_SE_C,TRDAR_SE_1,TRDAR_CD,TRDAR_CD_N,XCNTS_VALU,YDNTS_VALU,SIGNGU_CD,ADSTRD_CD,STDR_YM_CD,area,perimeter,x,y,geometry
0,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,201810,15720.885857,614.725259,126.956149,37.478185,"MULTIPOLYGON (((126.95719 37.47880, 126.95689 ..."
1,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,201810,9643.280304,478.077580,126.945127,37.481142,"MULTIPOLYGON (((126.94621 37.48127, 126.94614 ..."


## 추정 매출 상권/배후지

In [27]:
df_rev_14 = pd.read_csv('data/seoul-raw/상권/추정매출/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2014.csv', encoding='cp949')
df_rev_15 = pd.read_csv('data/seoul-raw/상권/추정매출/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2015.csv', encoding='cp949')
df_rev_16 = pd.read_csv('data/seoul-raw/상권/추정매출/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2016.csv', encoding='cp949')
df_rev_17 = pd.read_csv('data/seoul-raw/상권/추정매출/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2017.csv', encoding='cp949')
df_rev_18 = pd.read_csv('data/seoul-raw/상권/추정매출/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2018.csv', encoding='cp949')
df_rev_19 = pd.read_csv('data/seoul-raw/상권/추정매출/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv', encoding='cp949')
df_rev_20 = pd.read_csv('data/seoul-raw/상권/추정매출/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2020.csv', encoding='cp949')
df_rev_21 = pd.read_csv('data/seoul-raw/상권/추정매출/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2021.csv', encoding='cp949')

In [37]:
df_b_rev_14 = pd.read_csv('data/seoul-raw/상권/추정매출/2014년_우리마을가게_상권서비스_상권배후지-추정매출.csv', encoding='cp949')
df_b_rev_15 = pd.read_csv('data/seoul-raw/상권/추정매출/2015년_우리마을가게_상권서비스_상권배후지-추정매출.csv', encoding='cp949')
df_b_rev_16 = pd.read_csv('data/seoul-raw/상권/추정매출/2016년_우리마을가게_상권서비스_상권배후지-추정매출.csv', encoding='cp949')
df_b_rev_17 = pd.read_csv('data/seoul-raw/상권/추정매출/2017년_우리마을가게_상권서비스_상권배후지-추정매출.csv', encoding='cp949')
df_b_rev_18 = pd.read_csv('data/seoul-raw/상권/추정매출/2018년_우리마을가게_상권서비스_상권배후지-추정매출.csv', encoding='cp949')
df_b_rev_19 = pd.read_csv('data/seoul-raw/상권/추정매출/2019년_우리마을가게_상권서비스_상권배후지-추정매출.csv', encoding='cp949')
df_b_rev_20 = pd.read_csv('data/seoul-raw/상권/추정매출/2020년_우리마을가게_상권서비스_상권배후지-추정매출.csv', encoding='cp949')
df_b_rev_21 = pd.read_csv('data/seoul-raw/상권/추정매출/2021년_서울시 우리마을가게 상권분석서비스(상권배후지-추정매출).csv', encoding='cp949')

## 생활인구 상권/배후지

## 점포 상권/배후지

In [40]:
df_store = pd.read_csv('data/seoul-raw/상권/서울시 우리마을가게 상권분석서비스(상권-점포).csv', encoding='cp949')

In [46]:
df_store.nunique()

기준_년_코드           2
기준_분기_코드          4
상권_구분_코드          4
상권_구분_코드_명        4
상권_코드          1496
상권_코드_명        1496
서비스_업종_코드       100
서비스_업종_코드_명     100
점포_수            493
유사_업종_점포_수      510
개업_율             51
개업_점포_수          32
폐업_률             50
폐업_점포_수          59
프랜차이즈_점포_수       85
dtype: int64

## 아파트 상권/배후지

집객시설 상권/배후지

상주인구 상권/배후지

직장인구 상권/배후지

소득소비 배후지